# Live cu-inj-live-impact 

In [ ]:
# Setup directories, and convert dashboard notebook to a script for importing
#!./setup.bash

In [ ]:
#%load_ext autoreload
#%autoreload 2

In [2]:
from impact import evaluate_impact_with_distgen, run_impact_with_distgen
from impact.tools import isotime
from impact.evaluate import  default_impact_merit
from impact import Impact

from make_dashboard import make_dashboard
from get_vcc_image import get_live_distgen_xy_dist

import matplotlib as mpl

In [3]:
import pandas as pd

import json
import epics

import sys
import os
from time import sleep, time


import matplotlib.pyplot as plt

import matplotlib as mpl
mpl.use('Agg')

# Nicer plotting
#%config InlineBackend.figure_format = 'retina'

# Logging

In [4]:
PREFIX = 'lume-impact-live-demo'

In [5]:
import logging

# Gets or creates a logger
logger = logging.getLogger(PREFIX)  

# set log level
logger.setLevel(logging.INFO)

# define file handler and set formatter
file_handler = logging.FileHandler(f'{PREFIX}.log')
#formatter    = logging.Formatter('%(asctime)s : %(levelname)s : %(name)s : %(message)s')
formatter    = logging.Formatter(fmt="%(asctime)s :  %(name)s : %(message)s ", datefmt="%Y-%m-%dT%H:%M:%S%z")

# Add print to stdout
logger.addHandler(logging.StreamHandler(sys.stdout))

file_handler.setFormatter(formatter)

# add file handler to logger
logger.addHandler(file_handler)

# Configuration

Set up basic input sources and output path.

Depends on the `$LCLS_LATTICE` environmental variable pointing to a checked out copy of https://github.com/slaclab/lcls-lattice (protected)

In [6]:
DEBUG = True

ROOT = os.getcwd()

WORKDIR=os.path.expandvars('$SCRATCH')

In [7]:
# Directory for summary output
OUTPUTDIR = os.path.join(ROOT, 'output')

# Directory to output plots
PLOTDIR = os.path.join(ROOT, 'plot')

# Directory for archive files
APATH = os.path.join(ROOT, 'archive')

# PV -> Sim conversion table
CSV =  'pv_mapping/cu_inj_impact.csv'

DASHBOARD_KWARGS = {'outpath':PLOTDIR,
                    'screen1': 'YAG02',
                    'screen2': 'YAG03',
                    'screen3': 'OTR2'
                   }


VCC_DEVICE = 'CAMR:IN20:186' # LCLS
#BASEPV = 'CAMR:LT10:900' # FACET-II


# Base input files
CONFIG0 = {'impact_config': os.path.expandvars('$LCLS_LATTICE/impact/models/cu_inj/v0/ImpactT.yaml'),
          'distgen_input_file': os.path.expandvars('$LCLS_LATTICE/distgen/models/cu_inj/vcc_image/distgen.yaml')
#           'workdir':os.path.expandvars('$SCRATCH'),
#           'mpi_run': 'salloc --partition ard -N 1 -n {n} mpirun -n {n} --use-hwthread-cpus {command_mpi}',
#          'command_mpi': '/sdf/home/c/cmayes/impact/bin/ImpactTexe-mpi'   
         }

# Dummy file for distgen
DISTGEN_LASER_FILE = os.path.join(ROOT, 'distgen_laser.txt')

# Base settings
SETTINGS0 = {
 'distgen:t_dist:length:value': 4 * 1.65 ,     #  Inferred pulse stacker FWHM: 4 ps, converted to tukey length
 'distgen:n_particle': 1000,   
 'stop': 16.5, 
 'timeout': 10000,
 'header:Nx': 32,
 'header:Ny': 32,
 'header:Nz': 32,
 'total_charge': 0, # Set to 0 to turn off Space Charge for faster execution 
 'change_timestep_1:dt':4e-12, 
 
 # Parallel control
 'numprocs': 32,
    
 # SDF setup    
 'command': '/sdf/home/c/cmayes/impact/bin/ImpactTexe',    
 'command_mpi': '/sdf/home/c/cmayes/impact/bin/ImpactTexe-mpi',
 'mpi_run': 'salloc --partition ard -N 1 -n {n} /sdf/sw/gcc-4.8.5/openmpi-4.0.4/bin/mpirun -n {n} {command_mpi}'
   }

if DEBUG:
    logger.info('DEBUG MODE: Running without space charge for speed. ')
    SETTINGS0['distgen:n_particle'] =1000
    SETTINGS0['total_charge'] = 0

# EPICS -> Simulation settings

In [8]:
def get_live_settings(csv, base_settings={}):
    """
    Fetches live settings for all devices in the CSV table, and translates them to simulation inputs
     
    """
    df = pd.read_csv(csv)
    pv_names = list(df['device_pv_name'])
    
    df['pv_value'] = epics.caget_many(pv_names)
    df.set_index('device_pv_name', inplace=True)
    
    # Custom for enum
    #iris_enum = int(df.at['IRIS:LR20:130:CONFG_SEL', 'pv_value'])
    #df.at['IRIS:LR20:130:CONFG_SEL', 'pv_value'] = IRIS_DIAMETER_FROM_ENUM_IN_MM[iris_enum]
    
    # Assign impact
    df['impact_value'] = df['impact_factor']*df['pv_value'] 

    # Collect settings
    settings = base_settings.copy()
    settings.update(dict(zip(df['impact_name'], df['impact_value'])))

    # VCC image
    settings['distgen:xy_dist:file'] = get_live_distgen_xy_dist(filename=DISTGEN_LASER_FILE, vcc_device=VCC_DEVICE)
    
    return settings, df

# LIVE_SETTINGS, DF = get_live_settings(CSV, SETTINGS0)
#DF[['Variable', 'bmad_name', 'pv_value','pv_unit',  'device_min', 'device_max',  'impact_name', 'impact_factor', 'impact_unit',
#       'impact_description',  'impact_value']]

In [9]:
#CONFIG0

In [10]:
#%%time
#I0 = run_impact_with_distgen(LIVE_SETTINGS,  workdir=WORKDIR, **CONFIG0, verbose=True )

In [11]:
DO_TIMING = False

if DO_TIMING:
    import numpy as np
    import time
    results = []
    tlist = []
    nlist = 2**np.arange(1,8, 1)[::-1]
    for n in nlist:
        t1 = time.time()
        LIVE_SETTINGS['numprocs'] = n
        print(f'running wit {n}')
        result = run_impact_with_distgen(LIVE_SETTINGS,  workdir=WORKDIR, **CONFIG0, verbose=False )
        results.append(result)
        dt = time.time() - t1
        tlist.append(dt)
        print(n, dt)     
        
    tlist, nlist        

In [12]:
# %matplotlib inline
# I0.plot(['norm_emit_x'], y2=['sigma_x'], ylim=(0, 10e-6), ylim2=(0,2e-3), figsize=(16,9))

In [13]:
#I0.initial_particles.plot('x', 'y')

# Get live values, run Impact-T, make dashboard

In [14]:
# Patch this into the function below for the dashboard creation
def my_merit(impact_object, itime):
    # Collect standard output statistics
    merit0 = default_impact_merit(impact_object)
    # Make the dashboard from the evaluated object
    plot_file = make_dashboard(impact_object, itime=itime, **DASHBOARD_KWARGS)
    #print('Dashboard written:', plot_file)
    logger.info(f'Dashboard written: {plot_file}')
    
    # Assign extra info
    merit0['plot_file'] = plot_file    
    merit0['isotime'] = itime
    
    # Clear any buffers
    plt.close('all')

    return merit0

In [15]:
#my_merit(I0, '123')

In [16]:
def run1():
    dat = {}
    
    # Acquire settings
    mysettings, df = get_live_settings(CSV, SETTINGS0)
    itime = isotime()
    dat['isotime'] = itime
    
    # Record inputs
    dat['inputs'] = mysettings
    dat['config'] = CONFIG0
    dat['pv_mapping_dataframe'] = df.to_dict()
    
    
    logger.info(f'Acquired settings from EPICS at: {itime}')
    logger.info(f'Running evaluate_impact_with_distgen...')
    
    t0 = time()
    dat['outputs'] = evaluate_impact_with_distgen(mysettings,
                                       merit_f=lambda x: my_merit(x, itime),
                                       archive_path=APATH,
                                       workdir=WORKDIR,
                                       **CONFIG0 )
    logger.info(f'...finished in {(time()-t0)/60:.1f} min')
    fname = fname=f'{OUTPUTDIR}/{PREFIX}-{itime}.json'
    json.dump(dat, open(fname, 'w'))
    #print('Written:', fname)
    logger.info(f'Output written: {fname}')
    return dat
    

In [17]:
# Here are the results
#result = run1()
#result.keys()

Acquired settings from EPICS at: 2021-11-24T12:44:58-08:00
Running evaluate_impact_with_distgen...
Dashboard written: /sdf/home/c/cmayes/GitHub/lume-impact-live-demo/plot/lume-impact-live-demo-2021-11-24T12:44:58-08:00-dashboard.png
...finished in 5.7 min
Output written: /sdf/home/c/cmayes/GitHub/lume-impact-live-demo/output/lume-impact-live-demo-2021-11-24T12:44:58-08:00.json


In [ ]:
# Basic config
#result['config']

In [ ]:
# Simulation inputs
#result['inputs']

In [19]:
# Simulation outputs
#result['outputs']

# Show the plot 

In [21]:
#from IPython.display import Image
#Image(filename=result['outputs']['plot_file']) 

# loop it


In [ ]:
if __name__ == '__main__':
    while True:
        run1()